In [ ]:
  # Copyright [2022] [pop2piano]
  #      Licensed under the Apache License, Version 2.0 (the "License");
  #      you may not use this file except in compliance with the License.
  #      You may obtain a copy of the License at
  #         http://www.apache.org/licenses/LICENSE-2.0
  #      Unless required by applicable law or agreed to in writing, software
  #      distributed under the License is distributed on an "AS IS" BASIS,
  #      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  #      See the License for the specific language governing permissions and
  #      limitations under the License.

#@title Setup Environment
#@markdown - Install Pop2Piano and its dependencies (may take a few minutes).
#@markdown - We highly recommend to use a GPU runtime.
#@markdown - click: Runtime >> Change Runtime Type >> GPU
import os

model_path = '/content/model-1999-val_0.67311615.ckpt'

if not os.path.exists(model_path):
    # Model dosyası zaten mevcut değilse indir
    !wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt
else:
    print("Model zaten mevcut.")

!apt-get install -y fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning==1.8.4 essentia==2.1b6.dev1034 note-seq==0.0.5 pyFluidSynth==1.3.0

In [68]:
#@title Import codes

import os
import sys
sys.path.append("pop2piano")
import glob
import random
import shutil

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = OmegaConf.load("pop2piano/config.yaml")
wrapper = TransformerWrapper(config)
wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
model = "dpipqxiy"
wrapper.eval()
""

In [71]:
#@title Select Arranger(Composer)

composer = "composer1" #@param['composer1', 'composer2', 'composer3', 'composer4', 'composer5', 'composer6', 'composer7', 'composer8', 'composer9', 'composer10', 'composer11', 'composer12', 'composer13', 'composer14', 'composer15', 'composer16', 'composer17', 'composer18', 'composer19', 'composer20', 'composer21']

In [92]:
# Giriş ses dosyasının bulunduğu dosya yolu
input_audio_path = "/content/input/audio_to_midi.wav"

# Çıkış dosyalarının kaydedileceği dizin
output_directory = "/content/output"

#@title Upload audio file (Supports : An audio file like wav, mp3)
audio_file = input_audio_path

In [93]:
#@title Result

pm, composer, mix_path, midi_path = wrapper.generate(
    audio_path=audio_file,
    composer=composer,
    model=model,
    show_plot=False,
    save_midi=True,
    save_mix=True,
)

In [ ]:
if os.path.exists(midi_path):
    base_name = os.path.basename(midi_path)
    new_path = os.path.join(output_directory, base_name)

    # Hedef klasörde aynı isimde bir dosya varsa, uyarı ver
    if os.path.exists(new_path):
        os.remove(new_path)
        shutil.move(midi_path, new_path)
    else:
        shutil.move(midi_path, new_path)
else:
    print(f"Dosya bulunamadı: {midi_path}")

if os.path.exists(mix_path):
    base_name = os.path.basename(mix_path)
    new_path = os.path.join(output_directory, base_name)

    # Hedef klasörde aynı isimde bir dosya varsa, uyarı ver
    if os.path.exists(new_path):
        os.remove(new_path)
        shutil.move(mix_path, new_path)
    else:
        shutil.move(mix_path, new_path)
else:
    print(f"Dosya bulunamadı: {mix_path}")


In [ ]:
# Input Klasör içeriğini sil
success = True  # Başarı durumunu takip etmek için bir bayrak

delete_input = "/content/input"

for file in os.listdir(delete_input):
    file_path = os.path.join(delete_input, file)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        success = False

# Başarı durumuna göre yazı yazdır
if success:
    print("Seperated Input klasörü içeriği başarıyla silindi.")
else:
    print(f"Klasör içeriği silme başarısız {file_path}. Neden: {e}")